<a href="https://colab.research.google.com/github/skywalker0803r/Letter-of-Credit-Intelligent-Auxiliary-Semantic-Analysis-System/blob/main/notebook/%E4%B8%80%E7%A8%AE%E6%96%B0%E7%9A%84%E6%96%87%E6%9C%AC%E6%AF%94%E5%B0%8D%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 導入算法

In [12]:
import requests
url = 'https://gist.githubusercontent.com/skywalker0803r/7d1c5c3e8f11b6713a7994f08b62fbe9/raw/51992fcd154ea85b99f9332165fb5b4275139138/knn_nlp.py'
exec(requests.get(url).text)

# database

In [13]:
import pandas as pd
import re
data = pd.read_csv('/content/drive/MyDrive/NLP_dataset/Letter-of-Credit-Intelligent-Auxiliary-Semantic-Analysis-System/data/對應表/EXPNO對應表.csv')
def preprocess_45(x):
    x = str(x).upper() # 轉大寫字串
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    str.strip(x) # 移除左右空白
    # 去除多重空白
    x = x.replace('   ', ' ')
    x = x.replace('  ', ' ')
    # 出現在頭的 就不可能對到前後加空格的 這種情形要想想怎麼對照(加上左右空白)
    x = ' ' + x + ' '
    return x
data_45 = data['45A'].apply(preprocess_45)
data_45

0        MASS PVC RESIN B57QUANTITY 175 MT AT 1300 USD...
1        PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT PRI...
2         COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...
3        COMMODITY         AMOUNTPOLYESTER TEXTURED YA...
4        COMMODITY         AMOUNTPOLYESTER TEXTURED YA...
                              ...                        
8226     CFR KOBE JAPANVISCOSE RAYON STAPLE FIBER15DX4...
8227     CFR KOBE JAPANVISCOSE RAYON STAPLE FIBER15DX4...
8228     CFR KOBE JAPANVISCOSE RAYON STAPLE FIBER15DX4...
8229     187 MT PVC RESIN SUSPENSION GRADE S65D OF FOR...
8230     7000 MT IN 4X20 FT CONTAINERSPVC RESIN S60AS ...
Name: 45A, Length: 8231, dtype: object

# 建模

In [14]:
%%time
str_max_len = 100
model = coresystem(
    data_45.apply(lambda x:x[:str_max_len]).values.tolist(),
    str_max_len=str_max_len)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

CPU times: user 8min 12s, sys: 1.69 s, total: 8min 13s
Wall time: 8min 9s


# 先用訓練數據測試

In [33]:
%%time
test_data = np.random.choice(data_45.apply(lambda x:x[:str_max_len]).values.tolist(),size=3).tolist()
鄰居 = model.predict(test_data)
for i,j in zip(test_data,鄰居):
  print(i,'\n','上面是測試資料,下面是鄰居','\n',j)

 HDPE RESIN TAISOX 8040C312MT AT USD1255MTPACKING25KGS PER BAG NO PALLET ATTACHEDTOTALUSD39156000CIF 
 上面是測試資料,下面是鄰居 
  HDPE RESIN TAISOX 8040C312MT AT USD1255MTPACKING25KGS PER BAG NO PALLET ATTACHEDTOTALUSD39156000CIF
 DESCRIPTION OF GOODS QUANTITYMT UNIT PRICEUSDMT AMOUNTUSDPC RESINGRADE NO AC2820AH 15725  385000  6 
 上面是測試資料,下面是鄰居 
  DESCRIPTION OF GOODS QUANTITYMT UNIT PRICEUSDMT AMOUNTUSDPC RESINGRADE NO AC2820AH 15725  385000  6
 PURIFIED TEREPHTHALIC ACID 5016000KGTRADE TERMS F O B ANY PORT IN TAIWAN  
 上面是測試資料,下面是鄰居 
  PURIFIED TEREPHTHALIC ACID 5016000KGTRADE TERMS F O B ANY PORT IN TAIWAN 
CPU times: user 184 ms, sys: 0 ns, total: 184 ms
Wall time: 186 ms


# 再用外部測試數據測試

In [45]:
test_data2 = pd.read_csv('/content/drive/MyDrive/NLP_dataset/Letter-of-Credit-Intelligent-Auxiliary-Semantic-Analysis-System/data/測試數據/0927到1210.csv')
test_data2 = test_data2['45A'].apply(preprocess_45)
test_data2 = test_data2.apply(lambda x:x[:str_max_len]).values.tolist()
test_data2 = np.random.choice(test_data2,size=100).tolist()
鄰居 = model.predict(test_data2)

In [61]:
n = 100
for i,j in zip(test_data2[:n],鄰居[:n]):
  try:
    j = data_45[data_45==j].sample(1).values[0]
    print(i,'\n','上面是測試資料,下面是鄰居','\n',j)
  except:
    pass

 TERMS OF SALE CIF NHAVA SHEVA PORT INDIA288 MT OF PHTHALIC ANHYDRIDE AT USD 1230 PER MTAS PER PROFO 
 上面是測試資料,下面是鄰居 
   TERMS OF PRICE CFR AQABA PORTJORDAN COUNTRY OF ORIGIN TAIWAN84 MT OF DOP 
 FOB ANY SEAPORT IN TAIWANEVA 7350M VIRGIN RESIN 25MT AT USD257000MT  
 上面是測試資料,下面是鄰居 
  TAIRILIN BRAND POLYESTER FILM B GRADE  12U50U 40000 KGSCIF SHANGHAI PORT CHINA 
 FOB ANY SEAPORT IN TAIWANEVA 7350M VIRGIN RESIN 50MT AT USD350500MT  
 上面是測試資料,下面是鄰居 
  CIP SHENZHENCHINAABS RESIN GRADE NOAG15E196MTUSD240500MT 
 5250 MT PVC RESIN S60 AT USD 1860MTAS PER PROFORMA INVOICE NO A3X10211177 DATED 22OCT2021TERMS OF D 
 上面是測試資料,下面是鄰居 
  PVC SUSPENSION RESIN GRADE S65D54 MTAT USD1640 MTAS PER SC NO T21319CIF NHAVA SHEVA INDIA 
 PURIFIED ISOPHTHALIC ACID  40 MTSCIF NHAVA SHEVA INDIA  
 上面是測試資料,下面是鄰居 
  PURIFIED ISOPHTHALIC ACID   360 MTSCIF NHAVA SHEVA PORT INDIA 
 48 MT PROPYLENE COPOLYMER GRADE NO K9017 AT USD202000MTAS PER POC8103CIF VANCOUVER BC CANADA  
 上面是測試資料,下面是鄰居 
  120 MT PROPYLENE COPOLYME